In [1]:
%load_ext autoreload
%autoreload 2
import os
import torch
os.chdir("/home/yangkai/codegen-detection/")

In [2]:
class ModelConfig:
    plm_name = "microsoft/graphcodebert-base"
    cls_drop = 0.2
    cls_dim = 768
    temperature = 0.1
    trade_off = 5
from detection_models.bert import BertSupConCLS
model_config = ModelConfig()
model = BertSupConCLS(model_config)

Some weights of the model checkpoint at microsoft/graphcodebert-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
import torch
state_dict = torch.load("checkpoints/supcon_tf5.pt")
model.load_state_dict(state_dict)

<All keys matched successfully>

In [3]:
from simcse.simcse import RobertaForCL
model_args = {'pooler_type':'avg_first_last','temp':0.1,'mlp_only_train':True}
model = RobertaForCL.from_pretrained("microsoft/graphcodebert-base",**model_args)

{'pooler_type': 'avg_first_last', 'temp': 0.1, 'mlp_only_train': True}


Some weights of the model checkpoint at microsoft/graphcodebert-base were not used when initializing RobertaForCL: ['lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForCL from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForCL from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
torch.cuda.set_device(2)
model = model.cuda()

In [5]:
from collections import namedtuple
class DataConfig:
    config = namedtuple('config',['data_pth'])
    gen = config('dataset/csn_gpt_gen_test.pkl')
    human = config('dataset/csn_1k_hum_test.pkl')

from detection_dataset.bert import DualBertCLSDataset
dataset = DualBertCLSDataset(DataConfig(),"microsoft/graphcodebert-base",
                             "microsoft/graphcodebert-base",512,False,
                             input_key1='extracted_full_func',
                             input_key2='gpt_resampled_code_cleaned')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_2924481/701012159.py:8 in <module>                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_2924481/701012159.py'                       │
│                                                                                                  │
│ /home/yangkai/codegen-detection/detection_dataset/bert.py:88 in __init__                         │
│                                                                                                  │
│    85 │   │   │   │    input_key1='extracted_full_func',input_key2='question',frac=1.0,seed=44   │
│    86 │   │   super(DualBertCLSDataset,self).__init__()                                          │
│    87 │   │                                                                                      │
│ ❱  88 │   │   gen_data = select_data(data_cfg.gen)                                               │
│    89 │   │   human_data = select_data(data_cfg.human)                                           │
│    90 │   │                                                                                      │
│    91 │   │   self.all_data = pd.concat([gen_data, human_data])                                  │
│                                                                                                  │
│ /home/yangkai/codegen-detection/detection_dataset/bert.py:22 in select_data                      │
│                                                                                                  │
│    19                                                                                            │
│    20 def select_data(data_cfg):                                                                 │
│    21 │   data = pd.read_pickle(data_cfg.data_pth)                                               │
│ ❱  22 │   selected = data.label >= 0                                                             │
│    23 │   if hasattr(data_cfg,"domain"):                                                         │
│    24 │   │   selected &= data.source.isin(list(data_cfg.domain))                                │
│    25 │   if hasattr(data_cfg,"model_name"):                                                     │
│                                                                                                  │
│ /home/yangkai/anaconda3/envs/py38-torch113/lib/python3.8/site-packages/pandas/core/generic.py:59 │
│ 02 in __getattr__                                                                                │
│                                                                                                  │
│    5899 │   │   │   and self._info_axis._can_hold_identifiers_and_holds_name(name)               │
│    5900 │   │   ):                                                                               │
│    5901 │   │   │   return self[name]                                                            │
│ ❱  5902 │   │   return object.__getattribute__(self, name)                                       │
│    5903 │                                                                                        │
│    5904 │   def __setattr__(self, name: str, value) -> None:                                     │
│    5905 │   │   """                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'DataFrame' object has no attribute 'label'

In [6]:
def avg_pool(x,x_rep):
    length_mask = x != 1
    x_rep = x_rep * length_mask[:,:,None]
    return x_rep.sum(1) / (length_mask[:,:,None].sum(1) + 1e-7)

def cls_pool(x,x_rep):
    return x_rep[:,0,:]

In [10]:
from torch.utils.data.dataloader import DataLoader
from tqdm import tqdm
loader = DataLoader(dataset,64,shuffle=False,collate_fn=dataset.collate_fn,pin_memory=True)
all_rep = []
all_rep_resampled2 = []
all_label = []
with torch.no_grad():
    for x1,x2,y in tqdm(loader):
        x1,x2,y = x1.cuda(),x2.cuda(),y.cuda()
        # x1_rep = avg_pool(x1,model.plm(x1)[0])
        # x2_rep = avg_pool(x2,model.plm(x2)[0])
        x1_rep = model(x1,(x1!=1).to(torch.int),sent_emb=True).pooler_output
        x2_rep = model(x2,(x2!=1).to(torch.int),sent_emb=True).pooler_output
        all_rep.append(x1_rep)
        all_rep_resampled2.append(x2_rep)
        all_label.append(y)

100%|██████████| 16/16 [00:24<00:00,  1.53s/it]


In [11]:
all_rep = torch.cat(all_rep,dim=0)
all_rep_resampled = torch.cat(all_rep_resampled,dim=0)
all_rep_resampled2 = torch.cat(all_rep_resampled2,dim=0)
# all_rep_resampled3 = torch.cat(all_rep_resampled3,dim=0)
all_label = torch.cat(all_label,dim=0)

In [18]:
w,mu = torch.load('simcse/whiten_param_384_simcse_valid.pt')
def whiten(logits:torch.Tensor,W,mu):
    return (logits - mu.to(logits)) @ torch.tensor(W).to(logits)

all_rep = whiten(all_rep,w,mu)
all_rep_resampled = whiten(all_rep_resampled,w,mu)
all_rep_resampled2 = whiten(all_rep_resampled2,w,mu)
# all_rep_resampled3 = whiten(all_rep_resampled3,w,mu)

In [12]:
normed_all_rep = torch.nn.functional.normalize(all_rep,p=2)
normed_all_rep_resampled = torch.nn.functional.normalize(all_rep_resampled,p=2)
normed_all_rep_resampled2 = torch.nn.functional.normalize(all_rep_resampled2,p=2)
# normed_all_rep_resampled3 = torch.nn.functional.normalize(all_rep_resampled3,p=2)

In [13]:
sim = (normed_all_rep_resampled * normed_all_rep).sum(-1)
sim2 = (normed_all_rep_resampled2 * normed_all_rep).sum(-1)
# sim3 = (normed_all_rep_resampled3 * normed_all_rep).sum(-1)

In [17]:
score = torch.cat([sim[:,None],sim2[:,None]],dim=-1)
# score,_ = torch.max(score,dim=-1)
score = torch.mean(score,dim=-1)
# score = sim

In [15]:
score.shape,all_label.shape

(torch.Size([1000]), torch.Size([1000]))

In [18]:
from sklearn.metrics import average_precision_score,roc_auc_score
roc_auc = roc_auc_score(all_label.tolist(),score.tolist())
ap = average_precision_score(all_label.tolist(),score.tolist())
roc_auc,ap

(0.49985599999999997, 0.495952969310567)

In [27]:
thresh = 0.6
pred = score > thresh

In [28]:
((pred == all_label) & (pred == 1)).sum() / (pred == 1).sum()

tensor(0.8303, device='cuda:1')

In [29]:
((pred == all_label) & (pred == 1)).sum() / (all_label == 1).sum()

tensor(0.4766, device='cuda:1')

In [115]:
pred.sum()

tensor(571, device='cuda:7')

In [15]:
import openai
import os
openai.api_type = "azure"
openai.api_version = "2023-03-15-preview"
openai.api_base = 'https://yangkaigpt.openai.azure.com/'
openai.api_key = '7c9cb70f59b94f91b3c4981c5070f5a2'

In [48]:
test1 = dataset[0]['extracted_full_func']
test2 = dataset[0]['vicuna_resampled_code_cleaned2']

In [50]:
import torch
embed1 = torch.tensor(response['data'][0]['embedding'])
embed1 = torch.nn.functional.normalize(embed1,dim=0)

In [52]:
embed2 = torch.tensor(response['data'][0]['embedding'])
embed2 = torch.nn.functional.normalize(embed2,dim=0)

In [55]:
from tqdm import tqdm
all_rep = []
resample1_embed = []
resample2_embed = []
for i in tqdm(range(len(dataset))):
    code,resample1,resample2 = dataset[i]['extracted_full_func'],dataset[i]['vicuna_resampled_code_cleaned'],dataset[i]['vicuna_resampled_code_cleaned2']
    response = openai.Embedding.create(engine='text_emb_ada',input=code)
    all_rep.append(response['data'][0]['embedding'])

    response = openai.Embedding.create(engine='text_emb_ada',input=resample1)
    resample1_embed.append(response['data'][0]['embedding'])

    response = openai.Embedding.create(engine='text_emb_ada',input=resample2)
    resample2_embed.append(response['data'][0]['embedding'])

  3%|█                                        | 40/1540 [01:19<49:55,  2.00s/it]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_3837938/2075087267.py:10 in <module>                                              │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_3837938/2075087267.py'                      │
│                                                                                                  │
│ /home/yangkai/anaconda3/envs/py38-torch113/lib/python3.8/site-packages/openai/api_resources/embe │
│ dding.py:33 in create                                                                            │
│                                                                                                  │
│   30 │   │                                                                                       │
│   31 │   │   while True:                                                                         │
│   32 │   │   │   try:                                                                            │
│ ❱ 33 │   │   │   │   response = super().create(*args, **kwargs)                                  │
│   34 │   │   │   │                                                                               │
│   35 │   │   │   │   # If a user specifies base64, we'll just return the encoded string.         │
│   36 │   │   │   │   # This is only for the default case.                                        │
│                                                                                                  │
│ /home/yangkai/anaconda3/envs/py38-torch113/lib/python3.8/site-packages/openai/api_resources/abst │
│ ract/engine_api_resource.py:153 in create                                                        │
│                                                                                                  │
│   150 │   │   │   api_key, api_base, api_type, api_version, organization, **params               │
│   151 │   │   )                                                                                  │
│   152 │   │                                                                                      │
│ ❱ 153 │   │   response, _, api_key = requestor.request(                                          │
│   154 │   │   │   "post",                                                                        │
│   155 │   │   │   url,                                                                           │
│   156 │   │   │   params=params,                                                                 │
│                                                                                                  │
│ /home/yangkai/anaconda3/envs/py38-torch113/lib/python3.8/site-packages/openai/api_requestor.py:2 │
│ 16 in request                                                                                    │
│                                                                                                  │
│   213 │   │   request_id: Optional[str] = None,                                                  │
│   214 │   │   request_timeout: Optional[Union[float, Tuple[float, float]]] = None,               │
│   215 │   ) -> Tuple[Union[OpenAIResponse, Iterator[OpenAIResponse]], bool, str]:                │
│ ❱ 216 │   │   result = self.request_raw(                                                         │
│   217 │   │   │   method.lower(),                                                                │
│   218 │   │   │   url,                                                                           │
│   219 │   │   │   params=params,                                                                 │
│                                                                                                  │
│ /home/yangkai/anaconda3/envs/py38-torch113/lib/python3.8/site-packages/openai/api_requestor.py:5 │
│ 16 in request_raw                                          

In [60]:
all_rep = all_rep[:-1]

40

In [61]:
all_rep,resample1_embed,resample2_embed = torch.tensor(all_rep),torch.tensor(resample1_embed),torch.tensor(resample2_embed)

In [65]:
sim1 = (all_rep * resample1_embed).sum(-1)
sim2 = (all_rep * resample2_embed).sum(-1)

In [67]:
sim2

tensor([0.6947, 0.9016, 0.9927, 0.9622, 0.9716, 0.9620, 0.9878, 0.9695, 0.9926,
        0.9241, 0.9543, 0.9374, 0.9000, 0.9560, 0.9898, 0.9905, 0.9348, 0.9495,
        0.9774, 0.9583, 0.9941, 0.9839, 0.9307, 0.9332, 0.9023, 0.9383, 0.9553,
        0.9689, 0.9522, 0.9727, 0.9934, 0.9694, 0.9269, 0.9424, 0.9596, 0.9597,
        0.9890, 0.9276, 0.9577, 0.8820])